# SSD (Single Shot Multibox Detector)

In [1]:
import numpy as np

import chainer
import chainer.functions as F
from chainer import initializers
import chainer.links as L

## モデル

SSDは畳み込みニューラルネットワークを用いた高速な物体検出アルゴリズムである。  
VGG16をベースモデルとしたネットワーク構造は以下のとおりである。

<img src="image/ssd.png" style="width: 700px;">  
(引用 「SSD: Single Shot MultiBox Detector」)

SSDの特徴はマルチスケールに対応するため複数スケールの特徴マップを利用する。  
また、異なるアスペクト比の物体に対応するためアスペクト比ごとに識別する。

出力はそれぞれのセルに対してバウンディングボックスの位置・大きさと物体クラスの確信度（softmaxを作用させて確率）である。

（Non-Maximum SuppressionはSSDの出力である複数のバウンディングボックスから  
最終的な物体検出の出力となるバウンディングボックスを絞る手法であるが詳細は省略する。）

In [2]:
class _Normalize(chainer.Link):
    # conv4_3のみL2正規化後スケーリング(論文より)
    def __init__(self, n_channels, initial=0, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.add_param(
            'scale', n_channels,
            initializer=initializers._get_initializer(initial))

    def __call__(self, x):
        x = F.normalize(x, eps=self.eps, axis=1)
        scale = F.broadcast_to(self.scale[:, np.newaxis, np.newaxis], x.shape)
        return x * scale


class SSD300(chainer.Chain):
    # input size = 300
    """
    grids = (38, 19, 10, 5, 3, 1)
    """
    # 各グリッドにおけるアスペクト比
    aspect_ratios = ((2,), (2, 3), (2, 3), (2, 3), (2,), (2,))
    
    # 畳み込み層の重みの初期値
    conv_init = {
        'initialW': initializers.GlorotUniform(),
        'initial_bias': initializers.Zero(),
    }
    # スケーリング定数
    norm_init = {
        'initial': initializers.Constant(20),
    }

    def __init__(self, n_classes):
        self.n_classes = n_classes

        super(SSD300, self).__init__()
        with self.init_scope():
            # VGG16と同じ
            self.conv1_1 = L.Convolution2D(3, 64, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv1_2 = L.Convolution2D(64, 64, ksize=3, stride=1, pad=1, **self.conv_init)

            self.conv2_1 = L.Convolution2D(64, 128, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv2_2 = L.Convolution2D(128, 128, ksize=3, stride=1, pad=1, **self.conv_init)

            self.conv3_1 = L.Convolution2D(128, 256, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv3_2 = L.Convolution2D(256, 256, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv3_3 = L.Convolution2D(256, 256, ksize=3, stride=1, pad=1, **self.conv_init)

            self.conv4_1 = L.Convolution2D(256, 512, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv4_2 = L.Convolution2D(512, 512, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv4_3 = L.Convolution2D(512, 512, ksize=3, stride=1, pad=1, **self.conv_init)
            self.norm4 = _Normalize(512, **self.norm_init)

            self.conv5_1 = L.Convolution2D(512, 512, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv5_2 = L.Convolution2D(512, 512, ksize=3, stride=1, pad=1, **self.conv_init)
            self.conv5_3 = L.Convolution2D(512, 512, ksize=3, stride=1, pad=1, **self.conv_init)
            
            # 追加の畳み込み層
            self.conv6 = L.DilatedConvolution2D(
                                512, 1024, ksize=3, stride=1, pad=6, dilate=6, **self.conv_init)  # 間隔を開けた畳み込み層
            self.conv7 = L.Convolution2D(1024, 1024, ksize=1, stride=1, pad=0, **self.conv_init)

            self.conv8_1 = L.Convolution2D(1024, 256, ksize=1, stride=1, pad=0, **self.conv_init)
            self.conv8_2 = L.Convolution2D(256, 512, ksize=3, stride=2, pad=1, **self.conv_init)

            self.conv9_1 = L.Convolution2D(512, 128, ksize=1, stride=1, pad=0, **self.conv_init)
            self.conv9_2 = L.Convolution2D(128, 256, ksize=3, stride=2, pad=1, **self.conv_init)

            self.conv10_1 = L.Convolution2D(256, 128, ksize=1, stride=1, pad=0, **self.conv_init)
            self.conv10_2 = L.Convolution2D(128, 256, ksize=3, stride=1, pad=0, **self.conv_init)

            self.conv11_1 = L.Convolution2D(256, 128, ksize=1, stride=1, pad=0, **self.conv_init)
            self.conv11_2 = L.Convolution2D(128, 256, ksize=3, stride=1, pad=0, **self.conv_init)
            
            # バウンディングボックスの位置用
            self.loc = chainer.ChainList()  # (空のchainListインスタンス)
            # 物体クラスの確信度用
            self.conf = chainer.ChainList()

        for ar in self.aspect_ratios:
            # バウンディングボックスの数
            n = (len(ar) + 1) * 2
            # chainListインスタンスにlinkを追加
            # ボックスの位置・大きさ(x, y, w, h)
            self.loc.add_link(L.Convolution2D(
                None, n * 4, ksize=3, stride=1, pad=1, **self.conv_init))
            # 物体クラスの確信度(c1, c2, ..., cm, cm+1) (背景の確信度も)
            self.conf.add_link(L.Convolution2D(
                None, n * (self.n_classes + 1), ksize=3, stride=1, pad=1, **self.conv_init))

    def _features(self, x):
        ys = []
        # size: 300
        h = F.relu(self.conv1_1(x))
        h = F.relu(self.conv1_2(h))
        h = F.max_pooling_2d(h, ksize=2)
        # size: 150
        h = F.relu(self.conv2_1(h))
        h = F.relu(self.conv2_2(h))
        h = F.max_pooling_2d(h, ksize=2)
        # size: 75
        h = F.relu(self.conv3_1(h))
        h = F.relu(self.conv3_2(h))
        h = F.relu(self.conv3_3(h))
        h = F.max_pooling_2d(h, ksize=2)
        # size: 38
        h = F.relu(self.conv4_1(h))
        h = F.relu(self.conv4_2(h))
        h = F.relu(self.conv4_3(h))
        ys.append(self.norm4(h))
        h = F.max_pooling_2d(h, ksize=2)
        # size: 19
        h = F.relu(self.conv5_1(h))
        h = F.relu(self.conv5_2(h))
        h = F.relu(self.conv5_3(h))
        h = F.max_pooling_2d(h, ksize=3, stride=1, pad=1)
        # size: 19
        h = F.relu(self.conv6(h))
        h = F.relu(self.conv7(h))
        # size: 19
        ys.append(h)

        for i in range(8, 11 + 1):
            h = F.relu(self['conv{:d}_1'.format(i)](h))
            h = F.relu(self['conv{:d}_2'.format(i)](h))
            ys.append(h)
        # conv8_2 size: 9
        # conv9_2 size: 5
        # conv10_2 size: 3
        # conv11_2 size: 1

        return ys

    def _multibox(self, xs):
        """
        バウンディングボックスの総数
        n = (4, 6, 6, 6, 4, 4)
        h*w = (38, 19, 10, 5, 3, 1)^2
        h*w*n = 4*38^2 + ... + 4*1^2 = 8732
        """
        ys_loc = []
        ys_conf = []
        for i, x in enumerate(xs):
            loc = self.loc[i](x)   # (batch_size, n*4, h, w)
            loc = F.transpose(loc, (0, 2, 3, 1))  # (batch_size, h, w, *n4)
            loc = F.reshape(loc, (loc.shape[0], -1, 4))  # (batch_size, h*w*n, 4)
            ys_loc.append(loc)

            conf = self.conf[i](x)  # (batch_size, n*(n_classes+1), h, w)
            conf = F.transpose(conf, (0, 2, 3, 1))  # (batch_size, h, w, n*(n_classes+1))
            conf = F.reshape(
                conf, (conf.shape[0], -1, self.n_classes + 1))  # # (batch_size, h*w*n, n_classes+1)
            ys_conf.append(conf)

        y_loc = F.concat(ys_loc, axis=1)
        y_conf = F.concat(ys_conf, axis=1)

        return y_loc, y_conf

    def __call__(self, x):
        return self._multibox(self._features(x))

In [3]:
ssd = SSD300(10)

In [4]:
img = np.ones((1, 3, 300, 300), dtype=np.float32)
output = ssd(img)
print(output[0].shape, output[1].shape)

(1, 8732, 4) (1, 8732, 11)


## 損失関数

<img src="image/ssd_loss.png" style="width: 600px;">  

$N$: マッチしたバウンディングボックスの数  
$L_{conf}$: 物体クラスの確信度に対するロス  
$L_{loc}$: バウンディングボックスの位置・大きさに対するロス  
$\alpha$: 物体クラスの確信度に対するロスとバウンディングボックスの位置・大きさに対するロスの割合を制御するパラメータ  
$Pos, Neg$: ポジティブサンプル（物体が含まれる）、ネガティブサンプル（物体が含まれない）  
$cx, cy, w, h$: バウンディングボックスの中心座標と幅、高さ  
$x_{ij}^{p}$: バウンディングボックス$i$、カテゴリ$p$、真のボックス$j$であれば$1$、そうでないなら$0$  
$smooth_{L1}$: smooth L1誤差  
$l_{i}^{m}(d_{i}^{m})$: 予測したバウンディングボックス$i$の位置（中心座標と幅、高さ）  
$g_{i}^{m}$: 真のバウンディングボックス$j$の位置（中心座標と幅、高さ）  
$c_{i}^{p}$: バウンディングボックス$i$におけるカテゴリ$p$の確信度（$p=0$は背景）

In [5]:
def _elementwise_softmax_cross_entropy(x, t):
    # -log(cp_hat) = -cp + logsumexp(cp)
    p = F.reshape(
        F.select_item(F.reshape(x, (-1, x.shape[-1])), F.flatten(t)),
        t.shape)
    return F.logsumexp(x, axis=-1) - p


def _mine_hard_negative(loss, pos, k):
    # ハードネガティブがTrueになるarray（!pos == hard_negではない）
    # negの中でcp_hatが小さいpがhart_neg（kはハードネガティブの個数）
    rank = (loss * (pos - 1)).argsort(axis=1).argsort(axis=1)
    hard_neg = rank < (pos.sum(axis=1) * k)[:, np.newaxis]
    return hard_neg


def multibox_loss(x_loc, x_conf, t_loc, t_conf, k):
    # ポジティブ（物体が含まれる）
    pos = t_conf.data > 0
    # N=0ならloss=0
    if np.logical_not(pos).all():
        return 0, 0

    x_loc = F.reshape(x_loc, (-1, 4))
    t_loc = F.reshape(t_loc, (-1, 4))
    # smooth l1 loss  = huber_loss(a=1)
    # (ssdの出力はcx, cy, log(w), log(h))
    loss_loc = F.huber_loss(x_loc, t_loc, 1)
    loss_loc *= pos.flatten().astype(loss_loc.dtype)
    loss_loc = F.sum(loss_loc) / pos.sum()

    loss_conf = _elementwise_softmax_cross_entropy(x_conf, t_conf)
    hard_neg = _mine_hard_negative(loss_conf.data, pos, k)
    loss_conf *= xp.logical_or(pos, hard_neg).astype(loss_conf.dtype)
    loss_conf = F.sum(loss_conf) / pos.sum()

    return loss_loc, loss_conf